<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/Bitmex_Cryptos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
import time
import pandas as pd
import ta
import ccxt

# Set up Bitmex client
api_key = 'your_api_key'
api_secret = 'your_api_secret'
exchange = ccxt.bitmex({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
})

# Set up trading parameters
symbol = 'ETH/USD'
trade_qty = 1 # trade quantity in ETH
fee = 0.00075 # Bitmex trading fee

# Set up indicator parameters
ema1_period = 28
ema2_period = 48
stoch_rsi_period = 14

# Retrieve historical data
ohlcvs = exchange.fetch_ohlcv(symbol, timeframe='1h', since=exchange.milliseconds() - 24 * 60 * 60 * 1000)
df = pd.DataFrame(ohlcvs, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df.set_index('timestamp', inplace=True)
df.index = pd.to_datetime(df.index, unit='ms')
df = df.apply(pd.to_numeric)

# Create indicators
df['ema1'] = ta.trend.ema_indicator(df['close'], window=ema1_period)
df['ema2'] = ta.trend.ema_indicator(df['close'], window=ema2_period)
df['stoch_rsi'] = ta.momentum.stochrsi(df['close'], window=stoch_rsi_period)

# Set up trading variables
usd_balance = exchange.fetch_balance()['total']['USD']
eth_balance = exchange.fetch_balance()['total']['ETH']
usd_trade_size = trade_qty * df.iloc[-1]['close'] # use current close price for trade size
eth_trade_size = trade_qty

# Trading loop
while True:
    # Get current price and indicators
    ticker = exchange.fetch_ticker(symbol)
    current_price = ticker['last']
    current_ema1 = df['ema1'].iloc[-1]
    current_ema2 = df['ema2'].iloc[-1]
    current_stoch_rsi = df['stoch_rsi'].iloc[-1]
    
    # Buy signal
    if current_ema1 > current_ema2 and current_stoch_rsi < 0.8 and usd_balance > usd_trade_size:
        order = exchange.create_market_buy_order(symbol, eth_trade_size)
        usd_balance = exchange.fetch_balance()['total']['USD']
        eth_balance = exchange.fetch_balance()['total']['ETH']
        print(f'Bought {eth_trade_size} ETH at {current_price} USD')
    
    # Sell signal
    elif current_ema1 < current_ema2 and current_stoch_rsi > 0.2 and eth_balance > eth_trade_size:
        order = exchange.create_market_sell_order(symbol, eth_trade_size)
        usd_balance = exchange.fetch_balance()['total']['USD']
        eth_balance = exchange.fetch_balance()['total']['ETH']
        print(f'Sold {eth_trade_size} ETH at {current_price} USD')
    
    time.sleep(60) # wait for 1 minute before next iteration


The strategy of this code is a simple moving average crossover strategy with a momentum filter using the Stochastic RSI indicator. The idea is to buy when the short-term exponential moving average (EMA1) crosses above the longer-term exponential moving average (EMA2) and the Stochastic RSI is below 0.8, and to sell when EMA1 crosses below EMA2 and the Stochastic RSI is above 0.2. The code uses hourly candlestick data to generate trading signals and trades the ETH/USDT pair on the Binance exchange.